In [41]:
import sqlalchemy as db
from faker import Faker
from faker_food import FoodProvider
import random as random


In [42]:
faker= Faker()
faker.add_provider(FoodProvider)

In [43]:
faker.name()

'Victoria Lyons'

In [44]:
list_pays=[]
for i in range(100):
   list_pays.append(faker.country())
print(list_pays)

['Ethiopia', 'Micronesia', 'Dominica', 'Martinique', 'Somalia', 'Brazil', 'British Indian Ocean Territory (Chagos Archipelago)', 'Botswana', 'Nigeria', 'Guam', 'Reunion', 'Argentina', 'Nicaragua', 'Iran', 'Saint Martin', 'Cuba', 'Timor-Leste', 'Saint Vincent and the Grenadines', 'Guinea', 'Netherlands Antilles', 'Portugal', 'Croatia', 'Tunisia', 'Fiji', 'Netherlands', 'Uzbekistan', 'Mongolia', 'Ukraine', 'Mongolia', 'India', 'Guernsey', 'Guyana', 'Norfolk Island', 'Saint Kitts and Nevis', 'Zambia', 'United Arab Emirates', 'French Guiana', 'Taiwan', 'Nicaragua', 'Korea', 'Guatemala', 'Equatorial Guinea', 'Bahrain', 'Jordan', 'Saint Kitts and Nevis', 'Liechtenstein', 'Ecuador', 'Bolivia', 'Saint Lucia', 'Central African Republic', 'Central African Republic', 'Ghana', 'San Marino', 'Guam', 'Guinea-Bissau', 'Tunisia', 'Syrian Arab Republic', 'Iraq', 'Rwanda', 'Hungary', 'Belize', 'Saint Martin', 'Venezuela', 'Iraq', 'Suriname', 'Bolivia', 'Mongolia', 'Latvia', 'Iceland', 'Brazil', 'Korea',

In [45]:
faker.address()

'742 Christopher Locks Apt. 522\nTamaraborough, HI 25139'

In [46]:
faker.text()

'Thus perform attention fear choose chance evidence attorney. Fish perform rate drug.\nHour start need tree wrong choose organization. Two surface resource laugh unit sport manager.'

In [47]:
#fake.dish()
#fake.fruit()
#fake.spice()
#fake.vegetable()
#fake.ingredient()

In [48]:
# sqlite://<nohostname>/<path>
# where <path> is relative:
engine = db.create_engine("sqlite:///restaurant.db")

In [49]:
connection = engine.connect()
metadata = db.MetaData()


In [50]:


ingredient_table = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(), primary_key=True),
              db.Column('prix_achat', db.Float(), default=0.0),
              )



In [51]:


pays_table = db.Table('Pays', metadata,
              db.Column('pays', db.String(), primary_key=True),
              )



In [52]:


restaurant_table = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(), primary_key=True),
              db.Column('parking', db.Integer(),nullable=False),
              db.Column('espace_enfant', db.Integer(),default=0),
              db.Column('pmr', db.Integer(),default=0),
              db.Column('borne', db.Integer(),default=0),
              db.Column('capacite', db.Integer(),default=0),
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays"),nullable=False),
              )



In [53]:


stock_table = db.Table('Stock', metadata,
              db.Column('id_stock', db.Integer(),primary_key=True),
              db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('quantite', db.Integer(),default=0),
              )



In [54]:


item_table = db.Table('Item', metadata,
              db.Column('nom_item', db.String(),primary_key=True),
              db.Column('type', db.String(),nullable=False),
              db.Column('prix', db.Float(),default=0.00),
              )



In [55]:

carte_item_table = db.Table('CarteItem', metadata,
              db.Column('id_carte_item', db.Integer(),primary_key=True),
              db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [56]:
employes_table= db.Table ('Employes', metadata,
            db.Column('id_employes',db.Integer(),primary_key=True),
            db.Column('nom',db.String(),nullable=False),
            db.Column('role',db.String(),nullable=False),
            db.Column('id_superieur',db.Integer(),db.ForeignKey("Employes.id_employes")),
            db.Column('anciennete',db.Integer(),default=0),
            db.Column('compte_bancaire',db.Integer()),
            db.Column('note',db.Integer()),
            db.Column('adresse',db.String()),
            db.Column('code_postal',db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
            )

In [57]:
bulletin_paye_table = db.Table('BulletinPaye', metadata,
              db.Column('id_employes', db.Integer(),db.ForeignKey("Employes.id_employes"),primary_key=True),
              db.Column('date', db.String(),primary_key=True,nullable=False),
              db.Column('salaire', db.Float(),nullable=False),
               db.Column('poste', db.String(),nullable=False),
              )

In [58]:
recette_table = db.Table('Recette', metadata,
        db.Column('id_recette', db.Integer(),primary_key=True),
        db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
        db.Column('quantite_ingredient', db.Integer(),default=0)
        )

In [59]:
menu_table = db.Table('Menu', metadata,
        db.Column('nom_menu', db.String(),primary_key=True),
        db.Column('plat', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('dessert', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('boisson', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('prix', db.Integer(),default=0.00)
        )

In [60]:
carte_menu_table = db.Table('CarteMenu', metadata,
              db.Column('id_carte_menu', db.Integer(),primary_key=True),
              db.Column('nom_menu', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [61]:
addition_table = db.Table('Addition', metadata,
              db.Column('id_ticket', db.Integer(),primary_key=True),
              db.Column('heure', db.Integer(),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('id_employes', db.String(),db.ForeignKey("Employes.id_employes"),nullable=False),
              db.Column('somme', db.REAL(),default=0),
              db.Column('moyen_paiment', db.String(),nullable=False),
              )

In [62]:
panier_item_table = db.Table('PanierItem', metadata,
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),primary_key=True),
              db.Column('nom_item', db.Integer(),db.ForeignKey("Item.nom_item"),primary_key=True),
              db.Column('quantite_item', db.Integer(),default=0),
              )

In [63]:
panier_menu_table = db.Table('PanierMenu', metadata,
              db.Column('id_panier_menu', db.Integer(), primary_key=True),
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),nullable=False),
              db.Column('nom_menu', db.Integer(),db.ForeignKey("Menu.nom_menu"),nullable=False),
              db.Column('quantite_menu', db.Integer(),default=0),
              )

In [64]:
metadata.create_all(engine)

In [65]:
list_pays=[]
with engine.begin() as connection:
    for _ in range(10):
      insert_stmt = pays_table.insert().values(
        pays= faker.country(),
       )
      connection.execute(insert_stmt)

In [66]:
with engine.begin() as connection:
  pays_pk = connection.execute(db.select([pays_table.c.pays])).fetchall()

with engine.begin() as connection:
     for i in range(50):
       aleatoire=random.randint(0,len(pays_pk)-1)
       insert_stmt = restaurant_table.insert().values(
         code_postal= random.randint(10000,99999),
         capacite= random.randint(30,100),
         parking= random.randint(0,1),
         espace_enfant= random.randint(0,1),
         pmr=random.randint(0,1),
         borne=random.randint(0,1),
         pays=pays_pk[aleatoire][0],
        )
       connection.execute(insert_stmt)


In [67]:
print(pays_pk)
print(random.choice(pays_pk))
print(pays_pk[1])
print(pays_pk[1][0])
print(pays_pk[aleatoire][0])
print()

[('Western Sahara',), ('Niue',), ('South Africa',), ("Lao People's Democratic Republic",), ('Libyan Arab Jamahiriya',), ('Saint Helena',), ('Mozambique',), ('Albania',), ('Iceland',), ('Burkina Faso',)]
('Western Sahara',)
('Niue',)
Niue
Western Sahara



In [68]:
List_role=['manager','director','caissier','cuisinier']
List_employes=[]
for i in range(26):
    if i==0:
        List_employes.append(List_role[1])
    if i>0 and i<=3:
        List_employes.append(List_role[0])
    if i>4 and i<=17:
        List_employes.append(List_role[3])
    if i>16 :
        List_employes.append(List_role[2])
     
print(List_employes)

['director', 'manager', 'manager', 'manager', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier']


In [69]:
# with engine.begin() as connection:
#   employes_pk = connection.execute(db.select([employes_table.c.pays])).fetchall()
# e=0
# with engine.begin() as connection:
#     for i in range(26):
         
#       insert_stmt = employes_table.insert().values(
#         id_employes= i,
#         nom= faker.name(),
#         role=List_employes[i],
#         id_superieur = i+1,
#         anciennete= randint(0,10),
#         compte_bancaire=randint(10000000,99999999),
#         note=randint(0,10),
#         adresse=faker.address(),
#         code_postal=randint(10000,99999)
       
#        )
#       connection.execute(insert_stmt)

In [70]:
# with engine.begin() as connection:
#     for i in range(26):
#       insert_stmt = bulletin_paye_table.insert().values(
#         id_employes= i,
#         date= faker.date(),
#         salaire=randint(1200,2500),
#         poste =List_employes[i],
       
#        )
#       connection.execute(insert_stmt)

In [71]:
# with engine.begin() as connection:
#     for i in range(26):
#       insert_stmt = bulletin_paye_table.insert().values(
#         id_employes= i,
#         date= faker.date(),
#         salaire=randint(1200,2500),
#         poste =List_employes[i],
       
#        )
#       connection.execute(insert_stmt)

In [72]:
type_liste = ["boisson", "dessert", "plat"]
with engine.begin() as connection:
    for _ in range(30):
      insert_stmt = item_table.insert().values(
        nom_item = faker.unique.dish(),
        type = random.choice(type_liste),
        prix = round(random.uniform(2, 10),2),

      )
      connection.execute(insert_stmt)

In [73]:
with engine.begin() as connection:
    for _ in range(40):
      insert_stmt = ingredient_table.insert().values(
          nom_ingredient = faker.unique.ingredient(),
          prix_achat = round(random.uniform(0.2,8),2)

      )
      connection.execute(insert_stmt)

In [74]:
with engine.begin() as connection:
    item_pk = connection.execute(db.select([item_table.c.nom_item])).fetchall()

with engine.begin() as connection:
    ingredient_pk = connection.execute(db.select([ingredient_table.c.nom_ingredient])).fetchall()


with engine.begin() as connection:
    for i in range(len(item_pk)):
        ingredient_selectionne = []
        for nb_ingredient in range(random.randrange(1,6)):
            ingre = random.choice([ing for ing in ingredient_pk if ing not in ingredient_selectionne])[0]
            insert_stmt = recette_table.insert().values(
                
                nom_item = item_pk[i][0],
                nom_ingredient = ingre,
                
                quantite_ingredient = random.randrange(1,5),
            )
            ingredient_selectionne.append(ingre)
            
            connection.execute(insert_stmt)

In [75]:
with engine.begin() as connection:
    item_plat_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type=="plat")).fetchall()
with engine.begin() as connection:
    item_dessert_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type =="dessert")).fetchall()
with engine.begin() as connection:
    item_boisson_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type=="boisson")).fetchall()

with engine.begin() as connection:
    for _ in range(15):
      insert_stmt = menu_table.insert().values(
          nom_menu = faker.unique.company(),
          plat = random.choice(item_plat_pk)[0],
          dessert = random.choice(item_dessert_pk)[0],
          boisson = random.choice(item_boisson_pk)[0],
          prix = round(random.uniform(8,15),2)

      )
      connection.execute(insert_stmt)

In [76]:



with engine.begin() as connection:
    for i in range(len(pays_pk)):
        item_selectionne = []
        for nb_selectionne in range(random.randrange(10,15)):
            itemm = random.choice([it for it in item_pk if it not in item_selectionne])[0]
            insert_stmt = carte_item_table.insert().values(
                
                pays= pays_pk[i][0],
                nom_item = itemm,
               
            )
            item_selectionne.append(itemm)
            
            connection.execute(insert_stmt)

In [77]:
with engine.begin() as connection:
    menu_pk = connection.execute(db.select([menu_table.c.nom_menu])).fetchall()
with engine.begin() as connection:
    for i in range(len(pays_pk)):
        menu_selectionne = []
        for nb_selectionne in range(random.randrange(5,10)):
            menuu = random.choice([me for me in menu_pk if me not in menu_selectionne])[0]
            insert_stmt = carte_menu_table.insert().values(
                
                pays= pays_pk[i][0],
                nom_menu = menuu,
               
            )
            menu_selectionne.append(menuu)
            
            connection.execute(insert_stmt)

In [78]:
with engine.begin() as connection:
    ingredient_pk = connection.execute(db.select([ingredient_table.c.nom_ingredient])).fetchall()
with engine.begin() as connection:
    codepostal_pk = connection.execute(db.select([restaurant_table.c.code_postal])).fetchall()


with engine.begin() as connection:
    for i in range(len(codepostal_pk)):
        ingredient_selectionne = []
        for nb_ingredient in range(random.randrange(10,20)):
            ingre = random.choice([ing for ing in ingredient_pk if ing not in ingredient_selectionne])[0]
            insert_stmt = stock_table.insert().values(
                
                code_postal= codepostal_pk[i][0],
                nom_ingredient = ingre,
                
                quantite= random.randrange(1,15),
            )
            ingredient_selectionne.append(ingre)
            
            connection.execute(insert_stmt)